# NLP Spring 2021 - CA1

prepared by Sajjad Pakdaman Savoji - 810195517

## Import necessary libraries

In [1]:
from libs import *
from utils import *

[nltk_data] Downloading package wordnet to /home/savoji/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/savoji/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Read Data

In [2]:
hate_train_path = "Data/HateEval/train.csv"
hate_dev_path = "Data/HateEval/dev.csv"
hate_test_path = "Data/HateEval/test.csv"
off_train_path = "Data/OffenseEval/train.csv"

In [3]:
hate_train, hate_dev, hate_test, off_train = load_data(hate_train_path, hate_dev_path, hate_test_path, off_train_path)

### Preprocessing 

In [4]:
hate_train['text'] = hate_train['text'].apply(costum_preprocessing)
hate_dev['text'] = hate_dev['text'].apply(costum_preprocessing)
off_train['text'] = off_train['text'].apply(costum_preprocessing)
hate_test['text'] = hate_test['text'].apply(costum_preprocessing)

In [5]:
def train_function(hate_train, hate_dev):
#     prep_options = [lemma, stem]
#     prep_option_names = ['lemma', 'stem']
    model_options = [Laplace]
    model_option_names = ["Laplace"]
    tokenizer_options = [word_tokenize, char_tokenizer]
    tokenizer_option_names = ["word-level", "char-level"]
    
    records = []
    counter = 0
    for tknz, tknz_name in zip(tokenizer_options, tokenizer_option_names):
        for model, model_name in zip(model_options, model_option_names):
#             for opt, opt_name in zip(prep_options, prep_option_names):
            counter += 1
            opts = []
            opt_names = []

            train = prep(hate_train, opts, tknz)
            dev   = prep(hate_dev,   opts, tknz)
            lm_1_0, lm_1_1, lm_2_0, lm_2_1 = train_models(train, model)

            train_pred_labels_2 = predict_labels(train, lm_2_0, lm_2_1, 2)
            train_pred_labels_1 = predict_labels(train, lm_1_0, lm_1_1, 1)
            dev_pred_labels_2   = predict_labels(dev  , lm_2_0, lm_2_1, 2)
            dev_pred_labels_1   = predict_labels(dev  , lm_1_0, lm_1_1, 1)

            rec = evaluate_model(train.label, train_pred_labels_1, create_desc(tknz_name, model_name, opt_names, 1, 'train'))
            records.append(rec)
            rec = evaluate_model(train.label, train_pred_labels_2, create_desc(tknz_name, model_name, opt_names, 2, 'train'))
            records.append(rec)
            rec = evaluate_model(dev.label  , dev_pred_labels_1, create_desc(tknz_name, model_name, opt_names, 1, 'dev'))
            records.append(rec)
            rec = evaluate_model(dev.label  , dev_pred_labels_2, create_desc(tknz_name, model_name, opt_names, 2, 'dev'))
            records.append(rec)
            print(f'{counter}/{len(tokenizer_options)*len(model_options)}')
                
    return records

In [6]:
records = train_function(hate_train, hate_dev)
df = create_df_from_records(records, 'metrics')

|unigram|train|word-level|Laplace|basic prep|
|bigram|train|word-level|Laplace|basic prep|
|unigram|dev|word-level|Laplace|basic prep|
|bigram|dev|word-level|Laplace|basic prep|
1/2
|unigram|train|char-level|Laplace|basic prep|
|bigram|train|char-level|Laplace|basic prep|
|unigram|dev|char-level|Laplace|basic prep|
|bigram|dev|char-level|Laplace|basic prep|
2/2


Make data balanced using offensive train

In [6]:
dif = len(hate_train[hate_train.label==0]) - len(hate_train[hate_train.label==1])

In [7]:
additional = off_train[off_train.label==1][0:dif]

In [8]:
balanced_data = pd.concat([hate_train, additional])

In [9]:
len(balanced_data[balanced_data.label==0]) - len(balanced_data[balanced_data.label==1])

0

In [10]:
records = train_function(balanced_data, hate_dev)
# df = create_df_from_records(records, 'metrics-balanced')

|unigram|train|word-level|Laplace|basic prep|
|bigram|train|word-level|Laplace|basic prep|
|unigram|dev|word-level|Laplace|basic prep|
|bigram|dev|word-level|Laplace|basic prep|
1/2
|unigram|train|char-level|Laplace|basic prep|
|bigram|train|char-level|Laplace|basic prep|
|unigram|dev|char-level|Laplace|basic prep|
|bigram|dev|char-level|Laplace|basic prep|
2/2


In [10]:
def test_function(hate_train, hate_dev, hate_test):
    model_options = [Laplace]
    model_option_names = ["Laplace"]
    tokenizer_options = [word_tokenize]
    tokenizer_option_names = ["word-level"]
    
    for tknz, tknz_name in zip(tokenizer_options, tokenizer_option_names):
        for model, model_name in zip(model_options, model_option_names):
            opts = []
            opt_names = []

            train = prep(hate_train, opts, tknz)
            dev   = prep(hate_dev,   opts, tknz)
            test  = prep(hate_test,  opts, tknz)
            
            lm_1_0, lm_1_1, lm_2_0, lm_2_1 = train_models(train, model)

#             train_pred_labels_1 = predict_labels(train, lm_1_0, lm_1_1, 1)
#             dev_pred_labels_1   = predict_labels(dev,   lm_1_0, lm_1_1, 1)
            test_pred_labels_1  = predict_labels(test,  lm_1_0, lm_1_1, 1)

#             evaluate_model(train.label, train_pred_labels_1, create_desc(tknz_name, model_name, opt_names, 1, 'train'))
#             evaluate_model(dev.label  , dev_pred_labels_1, create_desc(tknz_name, model_name, opt_names, 1, 'dev'))
            
            test_final = pd.read_csv(hate_test_path)
            test_final.label = test_pred_labels_1
            test_final.to_csv('./Res/test_final.csv', index=False)

In [11]:
test_function(balanced_data, hate_dev, hate_test)